# IT - 542 Assignment - 7

In [1]:
import numpy as np
import random
import operator
import math
from sklearn.datasets import load_iris

np.set_printoptions(suppress=True)

## (1) Implement Fuzzy c-means clustering algorithm. Use IRIS data to evaluate performance of the algorithm. Compare your results with that of the in-built function.

In [2]:
# Load IRIS dataset
iris_data, iris_labels = load_iris(return_X_y=True)
print(iris_data.shape)
print(iris_labels.shape)

(150, 4)
(150,)


In [3]:
# Accuracy function for IRIS data
def accuracy_iris(cluster_labels, target_labels):
    # correct_pred = 0
    # for i in range(len(cluster_labels)):
    #     if cluster_labels[i] == target_labels[i]: correct_pred += 1
    correct_pred = np.where(cluster_labels == target_labels)[0]
    
    accuracy = (correct_pred.shape[0] / len(cluster_labels)) * 100
    return accuracy

### Custom implementation of Fuzzy C-Means

In [17]:
class FuzzyCMeans:

    def __init__(self, n_clusters, m=2, max_iter=300):
        self.n_clusters = n_clusters
        self.m = m
        self.max_iter = max_iter
        self.member_mat = None
        self.cluster_centers = []
        self.cluster_labels = []
    

    def init_membership_matrix(self):
        self.member_mat = np.zeros((self.n, self.n_clusters))
        for i in range(self.n):
            random.seed(202011032)
            init_mem_list = [np.random.random() for _ in range(self.n_clusters)]
            summation = sum(init_mem_list)
            init_mem_list = [x / summation for x in init_mem_list]
            
            flag = init_mem_list.index(max(init_mem_list))
            for j in range(0, len(init_mem_list)):
                if(j == flag):
                    init_mem_list[j] = 1
                else:
                    init_mem_list[j] = 0
            
            self.member_mat[i] = np.array(init_mem_list)

        # return member_mat
    

    def calculate_cluster_centers(self, data):
        clusters = self.member_mat.T
        self.cluster_centers = []
        for j in range(self.n_clusters):
            x = clusters[j]
            # x_raised = [p ** self.m for p in x]
            x_raised = np.power(x, self.m)
            denominator = sum(x_raised)
            temp_num = []
            # temp_num = np.multiply(data, x_raised)
            for i in range(self.n):
                data_point = data[i]
                # prod = data[i] * x_raised[i]
                prod = [x_raised[i] * val for val in data_point]
                temp_num.append(prod)
            # numerator = list(map(sum, list(zip(*temp_num))))
            numerator = np.sum(np.array(temp_num).T, axis=1)
            center = numerator / denominator
            self.cluster_centers.append(center)
        # return cluster_centers
    

    def update_member_mat(self, data):
        p = float(2 / (self.m - 1))
        for i in range(self.n):
            x = data[i]
            distances = [np.linalg.norm(np.array(list(map(operator.sub, data[i], self.cluster_centers[j])))) for j in range(self.n_clusters)]
            for j in range(self.n_clusters):
                denom = sum([math.pow(float(distances[j] / distances[c]), p) for c in range(self.n_clusters)])
                self.member_mat[i, j] = float(1 / denom)       
        # return member_mat
    
    
    def get_clusters(self):
        self.cluster_labels = []
        for i in range(self.n):
            max_val, label = max((val, idx) for (idx, val) in enumerate(self.member_mat[i]))
            self.cluster_labels.append(label)
        # return cluster_labels
    

    def fit(self, data):
        self.n = np.array(data).shape[0]
        self.init_membership_matrix()
        acc = []
        for curr in range(self.max_iter):
            self.calculate_cluster_centers(data)
            self.update_member_mat(data)
            self.get_clusters()
            
            acc.append(self.cluster_labels)
            
            if(curr == 0):
                print("Initial Cluster Centers:")
                print(np.array(self.cluster_centers))

        print("---------------------------")
        print("Partition matrix:")
        print(np.around(self.member_mat, decimals=4))
        #return cluster_labels, cluster_centers
        return self.cluster_labels, self.cluster_centers, acc

In [34]:
# Fit the IRIS data
fuzzy_cmeans_clusterer = FuzzyCMeans(n_clusters=3, m=3, max_iter=500)
cluster_labels, cluster_centers, acc = fuzzy_cmeans_clusterer.fit(iris_data)

Initial Cluster Centers:
[[5.72909091 3.08       3.47272727 1.05818182]
 [5.86909091 3.00363636 3.91818182 1.28545455]
 [5.965      3.1        3.93       1.275     ]]
---------------------------
Partition matrix:
[[0.9198 0.0471 0.033 ]
 [0.8226 0.1049 0.0725]
 [0.8335 0.0978 0.0687]
 [0.7983 0.1193 0.0824]
 [0.9002 0.0586 0.0412]
 [0.7279 0.1604 0.1117]
 [0.8315 0.0991 0.0694]
 [0.9726 0.0162 0.0112]
 [0.7251 0.162  0.113 ]
 [0.8471 0.0905 0.0623]
 [0.7947 0.1206 0.0847]
 [0.8916 0.0642 0.0442]
 [0.8069 0.114  0.0791]
 [0.7122 0.1676 0.1202]
 [0.6667 0.1921 0.1413]
 [0.6186 0.2196 0.1618]
 [0.7475 0.147  0.1056]
 [0.9202 0.047  0.0328]
 [0.6856 0.1852 0.1291]
 [0.8268 0.1017 0.0715]
 [0.8002 0.119  0.0808]
 [0.8487 0.0891 0.0622]
 [0.7719 0.1321 0.096 ]
 [0.8352 0.0987 0.0661]
 [0.798  0.1209 0.0811]
 [0.8175 0.1088 0.0737]
 [0.9115 0.0526 0.0359]
 [0.8952 0.0619 0.043 ]
 [0.8978 0.0602 0.042 ]
 [0.8353 0.0978 0.067 ]
 [0.8335 0.099  0.0675]
 [0.8149 0.1096 0.0755]
 [0.7328 0.1554 0.1

In [35]:
print(f"Cluster labels: {np.array(cluster_labels)}")
a = accuracy_iris(cluster_labels, iris_labels)
print(f"Accuracy: {a:.3f}")

Cluster labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 1 2 2 2 2
 2 2 1 2 2 2 2 2 1 2 1 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 2 2
 2 1]
Accuracy: 90.000


### Using `scikit-fuzzy` library for Fuzzy C-Means

In [40]:
import skfuzzy as fuzz

cl_centers, u_orig, _, _, _, _, _ = fuzz.cluster.cmeans(iris_data.T, 3, 3, error=0.005, maxiter=500)
print(cl_centers)
print("-----------------------------")
print("Partition Matrix:")
print(np.array([[a, b, c] for a, b, c in zip(u_orig[0], u_orig[1], u_orig[2])]))

[[5.00267073 3.40360833 1.49178815 0.25413933]
 [5.9108032  2.79154751 4.3795376  1.39688394]
 [6.69468326 3.03733703 5.55115454 2.03535714]]
-----------------------------
Partition Matrix:
[[0.91983125 0.04713757 0.03303118]
 [0.82268509 0.10485733 0.07245759]
 [0.83351346 0.09780365 0.06868289]
 [0.7983921  0.11923264 0.08237526]
 [0.90018416 0.05857425 0.04124159]
 [0.72796977 0.16035862 0.1116716 ]
 [0.83151023 0.09905207 0.06943769]
 [0.97263616 0.01616834 0.0111955 ]
 [0.72514792 0.16188617 0.11296591]
 [0.84718095 0.09049489 0.06232416]
 [0.79474822 0.120548   0.08470378]
 [0.89165831 0.06416299 0.0441787 ]
 [0.80691377 0.11397529 0.07911094]
 [0.71229528 0.16752635 0.12017837]
 [0.66668557 0.19200574 0.14130869]
 [0.61866522 0.21952071 0.16181407]
 [0.74748282 0.14692228 0.1055949 ]
 [0.92018609 0.04697529 0.03283862]
 [0.68566631 0.18517648 0.12915721]
 [0.8268296  0.10163686 0.07153354]
 [0.80023347 0.11892934 0.08083718]
 [0.84867738 0.08909024 0.06223239]
 [0.77192695 0.132

In [41]:
print("-----------------------------")
print(f"Cluster labels: {np.argmax(u_orig, axis=0)}")
print(f"Accuracy: {accuracy_iris(np.argmax(u_orig, axis=0), iris_labels)}")

-----------------------------
Cluster labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 1 2 2 2 2
 2 2 1 2 2 2 2 2 1 2 1 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 2 2
 2 1]
Accuracy: 90.0


## (2)  Implement Agglomerative clustering algorithm. Use IRIS data to evaluate performance of the algorithm. Compare your results with that of the in-built function.

### Custom implementation of Agglomerative clustering

In [42]:
class CustomAgglomerativeClustering:
    # By default uses euclidean distance measure
    def __init__(self, n_clusters: int = 2, linkage: str = 'single'):
        self.n_clusters_ = n_clusters
        self.labels_ = []
        self.__linkage = linkage
        self.__clusters = []
    

    def __single_linkage(self, data):
        # Every data point is a cluster at initial
        self.__clusters = np.expand_dims(data, axis=1).tolist()

        while len(self.__clusters) != self.n_clusters_:
            
            min_distance = cl_1 = cl_2 = math.inf
            # for every cluster (until second last element)
            for cl_idx, cluster in enumerate(self.__clusters[:-1]):
                # for each point in each cluster
                for point_id, point in enumerate(cluster): 
                    # compare with clusters after the current one
                    for cl_idx_2, cluster2 in enumerate(self.__clusters[(cl_idx + 1):]):
                        # go through every point in this prospective cluster as well
                        for point2_id, point2 in enumerate(cluster2):
                            # Check distance using single linkage and euclidean distance
                            clust_diff = np.array(point) - np.array(point2)
                            if np.linalg.norm(clust_diff) < min_distance:  
                                min_distance = np.linalg.norm(clust_diff)
                                # this will be used at the end to figure out which cluster to merge with which
                                cl_1 = cl_idx
                                # this cluster will be destroyed by the end
                                cl_2 = (cl_idx + 1) + cl_idx_2
            
            # merge clusters
            print(cl_1,' | ', cl_2, ' | ', min_distance)
            self.__clusters[cl_1].extend(self.__clusters[cl_2])
            # remove the cluster which is now merged in another cluster
            self.__clusters.pop(cl_2)


    def __get_labels(self, data):
        # Call after clustering is done, otherwise will raise an error
        if len(self.__clusters) == 0:
            raise ValueError("Clusters list is empty. Run clustering algorithm first.")

        for data_pt in data.tolist():
            if data_pt in self.__clusters[0]:
                self.labels_.append(0)
            elif data_pt in self.__clusters[1]:
                self.labels_.append(1)
            else:
                # data_pt will be in self.__clusters[2]
                self.labels_.append(2)


    
    def fit(self, data: np.ndarray):
        if self.__linkage == 'single':
            self.__single_linkage(data)
        self.__get_labels(data)


In [43]:
agglomerative_clusterer = CustomAgglomerativeClustering(n_clusters=3, linkage='single')
agglomerative_clusterer.fit(iris_data)

101  |  142  |  0.0
7  |  39  |  0.09999999999999964
0  |  17  |  0.09999999999999998
9  |  33  |  0.1
125  |  129  |  0.10000000000000009
10  |  45  |  0.10000000000000053
0  |  37  |  0.14142135623730917
4  |  35  |  0.14142135623730925
18  |  20  |  0.14142135623730928
0  |  4  |  0.1414213562373093
26  |  27  |  0.1414213562373093
48  |  84  |  0.1414213562373093
71  |  72  |  0.1414213562373093
105  |  125  |  0.1414213562373093
7  |  32  |  0.14142135623730948
17  |  37  |  0.14142135623730956
0  |  6  |  0.14142135623730964
0  |  37  |  0.14142135623730964
1  |  7  |  0.14142135623730964
3  |  35  |  0.14142135623730964
22  |  23  |  0.14142135623730964
65  |  75  |  0.14142135623730964
76  |  77  |  0.14142135623730964
107  |  116  |  0.14142135623730964
2  |  3  |  0.14142135623730978
1  |  32  |  0.14142135623730986
1  |  8  |  0.1414213562373099
0  |  20  |  0.14142135623730995
1  |  20  |  0.14142135623730995
42  |  69  |  0.14142135623730995
44  |  54  |  0.141421356237309

In [44]:
print(agglomerative_clusterer.labels_)
print(f"Accuracy: {accuracy_iris(agglomerative_clusterer.labels_, iris_labels)}")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Accuracy: 68.0


### Using `sklearn` library's Agglomerative Clustering

In [46]:
from sklearn.cluster import AgglomerativeClustering
sk_clusterer = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='single')
print(sk_clusterer.fit_predict(iris_data))
print(f"Accuracy: {accuracy_iris(sk_clusterer.labels_, iris_labels)}")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
Accuracy: 1.3333333333333335
